In [1]:
import sys; sys.path.append('..')

In [2]:
import math
from os import path
import random

In [3]:
# import networkx as nx
# from matplotlib import pyplot as plt

In [4]:
from graph import Graph
from graph.disjointsets import DisjointSets
from graph.reader import read_problem
from graph.steiner import prunning_mst
from graph.util import has_cycle, how_many_components

In [5]:
from base.binary.combiner import crossover_1point, crossover_2points, crossover_uniform
# from base.chromosome import random_binary
from base.condition import BestKnownReached, BestSolutionKnowReached, Stagnation
from base.customevol import SteinerEvolution as Evolution
from base.customevol import SteinerPopulation as Population
from base.mutate import flip_onebit
from base.normalization import normalize
from base.pickers import random_picker
from base.selector import roullete
from base.tracker import DataTracker
from base.util import display, update_best, update_generation

from treetools import Eval, Converter

In [6]:
from draw import hierarchy_pos

---

In [7]:
# problema = path.join('..', 'datasets', 'ORLibrary', 'steinc1.txt')
problema = path.join('..', 'datasets', 'ORLibrary', 'steinb16.txt')

STPG = read_problem(problema)

In [8]:
best_known_solution = 127

In [9]:
STPG.nro_nodes

100

In [10]:
STPG.nro_edges

200

In [11]:
STPG.nro_terminals

17

In [12]:
print(STPG.terminals)

{97, 37, 69, 72, 9, 76, 60, 81, 82, 17, 20, 21, 54, 87, 25, 59, 28}


In [13]:
terminals = set(STPG.terminals)

def is_terminal(vertice):
    return vertice in terminals

In [14]:
def prunning(treegraph):
    
    to_eliminate = set()
    
    for v in treegraph.vertices:
        degree = treegraph.degree(v)
        if (not is_terminal(v)) and (degree == 1):
            to_eliminate.add(v)
        elif (degree == 0) and (not is_terminal(v)) :
            to_eliminate.add(v)
    
    checkbefore = set()
    
    while to_eliminate:
        while to_eliminate:
            v = to_eliminate.pop()
            for adj in treegraph.adjacent_to(v):
                checkbefore.add(adj)
            treegraph.remove_node(v)

        while checkbefore:
            u = checkbefore.pop()
            if (not is_terminal(u)) and (treegraph.degree(u) == 1):
                to_eliminate.add(u)
                
    return treegraph   

In [15]:
def random_treegraph_chromosome(STPG):
    
    graph = STPG.graph
    terminals = set(STPG.terminals)
    
    alledges = [(v, u, graph.weight(v,u)) for v, u in graph.gen_undirect_edges()]
    random.shuffle(alledges) # random component

    DS = DisjointSets()
    subgraph = Graph()
    total_cost = 0
    
    for v in graph.vertices:
        DS.make_set(v)

    while alledges :
        v, u, weight = alledges.pop()

        if DS.find(v) != DS.find(u):
            DS.union(v, u)
            total_cost += weight
            subgraph.add_edge(v, u, weight=weight)
            terminals.discard(v)
            terminals.discard(u)

    return prunning(subgraph)

In [16]:
population_size = 50
mutation_rate = 0.2
cx_uniform_rate = 0.5

nro_repetition = 500

In [17]:
converter = Converter(STPG)
fitness_func = Eval(STPG)

In [18]:
population = (Population(chromosomes=[random_treegraph_chromosome(STPG) for _ in range(population_size)],
                        eval_function=fitness_func,
                        maximize=True)
                        .evaluate()
                        .callback(normalize)
                        .callback(update_best))

In [19]:
kapsalis = (Evolution()
            .evaluate()
            .callback(normalize)
            .callback(update_best)
            .select(selection_func=roullete)
            .crossover(combiner=crossover_2points, 
                       parent_picker=random_picker)
            .mutate(mutate_function=flip_onebit, 
                    probability=mutation_rate)
            .callback(update_generation)
            .callback(display, every=100))

In [20]:
def prunning__(individuo):    
    if individuo.qtd_partitions == 1:
        individuo.chromosome = prunning(individuo.chromosome)

    return individuo

In [21]:
hybridi = (Evolution()
           .map(converter.treegraph2binary)
           .repeat(kapsalis, n=nro_repetition)
           .map(converter.binary2treegraph)
           .map(prunning__)
          )

In [22]:
%%time

with Stagnation(interval=1_000), \
        BestSolutionKnowReached(global_optimum=best_known_solution, STPG=STPG):
    result = population.evolve(hybridi, n=20)

Population b0a4e6 | size 50 | generation 100
Population b0a4e6 | size 50 | generation 200
Population b0a4e6 | size 50 | generation 300
Population b0a4e6 | size 50 | generation 400
Population b0a4e6 | size 50 | generation 500
Population b0a4e6 | size 50 | generation 600
Population b0a4e6 | size 50 | generation 700
Population b0a4e6 | size 50 | generation 800
Population b0a4e6 | size 50 | generation 900
Population b0a4e6 | size 50 | generation 1000
Population b0a4e6 | size 50 | generation 1100
Population b0a4e6 | size 50 | generation 1200
Population b0a4e6 | size 50 | generation 1300
Population b0a4e6 | size 50 | generation 1400
Population b0a4e6 | size 50 | generation 1500
Wall time: 1min 53s


In [23]:
result.documented_best.cost

156

In [24]:
result.documented_best.qtd_partitions

1

In [25]:
result.evaluate()

for p in result:
    print(p.cost)

186
183
191
194
199
188
195
186
194
203
183
185
196
196
191
185
197
186
189
191
187
287
294
187
186
188
189
184
191
180
184
195
189
182
194
187
179
193
183
192
185
189
181
192
198
195
203
200
302
189
